To get started you need to link your google drive with google colab in the files tab to the left.

Upload [this](https://github.com/gemenerik/nanoflownet-cnns/blob/main/pretrained_models/nanoflownet/nanoflownet_unquantized.tflite) file (the neural network .tflite) to your main google drive directory.

In the following code it is used in the Load Model part. In case you want to choose a different directory:

```
# Load TFLite model and allocate tensors
tflite_model_path = '/content/drive/MyDrive/nanoflownet_unquantized.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
```


In [1]:
# @title Imports:
import tensorflow as tf
import numpy as np
from IPython.display import display, Image, Javascript
import base64
from PIL import Image as PILImage
from io import BytesIO
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
from google.colab.output import eval_js

In [14]:
# @title Define Functions:
def take_photo_pair(quality=0.8, interval=20):  # interval is in milliseconds
    js_code = f'''
    async function takePhoto(quality, interval) {{
      const video = document.createElement('video');
      video.style.display = 'none';  // Set display to 'none' to hide the video element
      const stream = await navigator.mediaDevices.getUserMedia({{ video: true }});

      video.srcObject = stream;
      await video.play();

      return [video, stream];
    }}

    async function captureFrames(video, quality, interval, resolve) {{
      const frames = [];
      let frameCount = 0;

      function captureFrame() {{
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);

        const dataUrl = canvas.toDataURL('image/jpeg', quality);
        frames.push(dataUrl);
        frameCount += 1;

        if (frameCount >= 2) {{
          resolve(frames);
        }}
      }}

      setInterval(captureFrame, interval);
    }}

    takePhoto({quality}).then(([video, stream]) => {{
      return new Promise((resolve) => {{
        captureFrames(video, {quality}, {interval}, resolve);
      }});
    }});
    '''

    result = eval_js(js_code)

    return result


def make_colorwheel():
    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR
    colorwheel = np.zeros((ncols, 3))
    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.floor(255 * np.arange(0, RY) / RY)
    col = col + RY
    # YG
    colorwheel[col:col + YG, 0] = 255 - np.floor(255 * np.arange(0, YG) / YG)
    colorwheel[col:col + YG, 1] = 255
    col = col + YG
    # GC
    colorwheel[col:col + GC, 1] = 255
    colorwheel[col:col + GC, 2] = np.floor(255 * np.arange(0, GC) / GC)
    col = col + GC
    # CB
    colorwheel[col:col + CB, 1] = 255 - np.floor(255 * np.arange(CB) / CB)
    colorwheel[col:col + CB, 2] = 255
    col = col + CB
    # BM
    colorwheel[col:col + BM, 2] = 255
    colorwheel[col:col + BM, 0] = np.floor(255 * np.arange(0, BM) / BM)
    col = col + BM
    # MR
    colorwheel[col:col + MR, 2] = 255 - np.floor(255 * np.arange(MR) / MR)
    colorwheel[col:col + MR, 0] = 255
    return colorwheel


def flow_uv_to_colors(u, v, convert_to_bgr=False):
    flow_image = np.zeros((u.shape[0], u.shape[1], 3), np.uint8)
    colorwheel = make_colorwheel()  # shape [55x3]
    ncols = colorwheel.shape[0]
    rad = np.sqrt(np.square(u) + np.square(v))
    a = np.arctan2(-v, -u) / np.pi
    fk = (a + 1) / 2 * (ncols - 1)
    k0 = np.floor(fk).astype(np.int32)
    k1 = k0 + 1
    k1[k1 == ncols] = 0
    f = fk - k0
    for i in range(colorwheel.shape[1]):
        tmp = colorwheel[:, i]
        col0 = tmp[k0] / 255.0
        col1 = tmp[k1] / 255.0
        col = (1 - f) * col0 + f * col1
        idx = rad <= 1
        col[idx] = 1 - rad[idx] * (1 - col[idx])
        col[~idx] = col[~idx] * 0.75  # out of range
        ch_idx = 2 - i if convert_to_bgr else i
        flow_image[:, :, ch_idx] = np.floor(255 * col)
    return flow_image


def flow_to_color(flow_uv, clip_flow=None, convert_to_bgr=False, flow_norm=None):
    assert flow_uv.ndim == 3, "input flow must have three dimensions"
    assert flow_uv.shape[2] == 2, "input flow must have shape [H,W,2]"
    if clip_flow is not None:
        flow_uv = np.clip(flow_uv, 0, clip_flow)
    u = flow_uv[:, :, 0]
    v = flow_uv[:, :, 1]

    if flow_norm is not None:
        assert flow_norm > 0
    else:
        rad = np.sqrt(np.square(u) + np.square(v))
        rad_max = np.max(rad)
        epsilon = 1e-5
        flow_norm = rad_max + epsilon
    u = u / flow_norm
    v = v / flow_norm
    return flow_uv_to_colors(u, v, convert_to_bgr)

In [3]:
# @title Load Model:
# Load TFLite model and allocate tensors
tflite_model_path = '/content/drive/MyDrive/nanoflownet_unquantized.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
# @title Execute NanoFlowNet:
# Continuously capture and display images
try:
    while True:
        start_time = time.time()

        # Capture image pair
        frames = take_photo_pair()

        # Process and display each frame
        for i, frame_data in enumerate(frames):
            # Decode base64 image
            img_data = base64.b64decode(frame_data.split(',')[1])

            # Convert to grayscale and resize
            frame = PILImage.open(BytesIO(img_data)).convert('L')
            frame = frame.resize((input_details[0]['shape'][2], input_details[0]['shape'][1]))

            # Preprocess the frame
            preprocessed_frame = np.array(frame)[:, :, np.newaxis]
            preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0).astype(input_details[0]['dtype'])

            # Run inference
            interpreter.set_tensor(input_details[0]['index'], preprocessed_frame)
            interpreter.invoke()

            output_data = interpreter.get_tensor(output_details[0]['index'])
            output_image = np.squeeze(output_data)

            # Visualizing optical flow using provided functions
            u = output_image[:, :, 0]
            v = output_image[:, :, 1]
            flow_image = flow_to_color(output_image, convert_to_bgr=True)

            # Clear the previous output
            clear_output(wait=True)

            # Display the flow image using matplotlib
            plt.imshow(flow_image)
            plt.axis('off')
            plt.show()

        # Control the frame rate
        elapsed_time = time.time() - start_time
        time_to_sleep = max(0, 1 / 10 - elapsed_time)
        time.sleep(time_to_sleep)

except Exception as err:
    print(str(err))